In [1]:
#             this_el = s2.CellUnion(map(lambda x: s2.CellId.from_token(x), element.get('cover16s')))
#             dict_el = s2.CellUnion(map(lambda x: s2.CellId.from_token(x), l_dict[token].get('cover16s')))

#             intersection = list(map(lambda x: x.to_token(), s2.CellUnion.get_intersection(this_el, dict_el).cell_ids()))
#             difference = list(map(lambda x: x.to_token(), s2.CellUnion.get_difference(this_el, dict_el).cell_ids()))
            
#             if len(intersection) < len(difference):
                
#                 bad_count +=1
#                 print(element.get('name'))
#                 print(l_dict[token].get('name'))
                
#             if s2.CellUnion.get_intersection(a,b).cell_ids():
#                 bad_count +=1
#             if (a.get_rect_bound().interior_contains((b.get_rect_bound().get_center())) or b.get_rect_bound().interior_contains((a.get_rect_bound().get_center()))):
#                 bad_count +=1
#                 print(element.get('name'))
#                 print(l_dict[token].get('name'))
#                 if bad_count > 10:
#                     return

          

#             for key in n_dict[name].keys():
#                 if not key in ['name', 'type', 'nameisid', 'nodelist']:
#                     this_el = s2.CellUnion(map(lambda x: s2.CellId.from_token(x), element.get(key)))
#                     dict_el = s2.CellUnion(map(lambda x: s2.CellId.from_token(x), n_dict[name].get(key)))
#                     new_cell_list = list(map(lambda x: x.to_token(), s2.CellUnion.get_union(this_el, dict_el).cell_ids()))
#                     n_dict[name][key] = new_cell_list


#             if len(list(map(lambda x: x.to_token(), this_el_16.cell_ids()))) + len(list(map(lambda x: x.to_token(), dict_el_16.cell_ids()))) - len(list(map(lambda x: x.to_token(), s2.CellUnion.get_intersection(this_el_16, dict_el_16).cell_ids()))) != len(new_cell_list_16):
#                 print("broke")
#                 return
                
            

In [2]:
import json
import s2sphere as s2
import copy
import uuid

In [3]:
with open('oldjsonscells/cells_college.json', encoding='utf8') as read_file:
    cells_college = json.load(read_file)
with open('oldjsonscells/cells_college_bad.json', encoding='utf8') as read_file:
    cells_college_bad = json.load(read_file)
with open('oldjsonscells/cells_university.json', encoding='utf8') as read_file:
    cells_university = json.load(read_file)
with open('oldjsonscells/cells_university_bad.json', encoding='utf8') as read_file:
    cells_university_bad = json.load(read_file)
with open('oldjsonscells/cells_school_bad.json', encoding='utf8') as read_file:
    cells_school_bad = json.load(read_file)
    
cells_school = []
for i in range(1000):
    with open('oldjsonscells/cells_school_' + str(i) + '.json', encoding='utf8') as read_file:
        cells_school.extend(json.load(read_file))

In [4]:
len(cells_college)

30193

In [5]:
len(cells_university)

35512

In [6]:
def get_union_preserve_custom(a,b,key):
    c = s2.CellUnion.get_union(a, b).cell_ids()
    running_union = s2.CellUnion()
    for cellId in copy.deepcopy(c):
        temp_union = s2.CellUnion([cellId])
        min_target = 10
        if '10' in key:
            min_target = 10
        elif '11' in key:
            min_target = 11
        elif '12' in key:
            min_target = 12
        elif '13' in key:
            min_target = 13
        elif '14' in key:
            min_target = 14
        elif '15' in key:
            min_target = 15
        elif '16' in key:
            min_target = 16
        if cellId.level() < min_target:
            temp_union = s2.CellUnion(temp_union.denormalize(min_target,1),raw=False)
        running_union = s2.CellUnion.get_union(temp_union, running_union,raw=False)
    return running_union

In [7]:
def remove_name_is_id(all_cells_lists):
    all_list = []
    int_list = []
    for cell_list in all_cells_lists:
        for element in cell_list:
            try:
                int(element.get('name'))
                int_list.append(element)
            except ValueError:
                all_list.append(element)
                continue
    return all_list, int_list

In [8]:
# total_cells, name_is_id_list = remove_name_is_id([cells_college, cells_college_bad, cells_university, cells_university_bad])
total_cells = [cells_college, cells_college_bad]

In [9]:
def namecheck(cell_list, name):
    for cell in cell_list:
        if name in cell['name']:
            print(cell)

In [10]:
def cleanInitialSubcategories(all_cells_list):
    new_list = []
    for all_cells in all_cells_list:
        for cell in all_cells:
            newcell = copy.deepcopy(cell)
            for key in newcell.keys():
                if not key in ['name', 'type', 'nameisid', 'nodelist']:
                    this_el = s2.CellUnion(map(lambda x: s2.CellId.from_token(x), newcell.get(key)))
                    dict_el = s2.CellUnion(map(lambda x: s2.CellId.from_token(x), newcell.get(key)))
                    new_cell_list = list(map(lambda x: x.to_token(), get_union_preserve_custom(this_el, dict_el, key).cell_ids()))
                    newcell[key] = new_cell_list
            new_list.append(newcell)
    return new_list

In [11]:
total_cells = cleanInitialSubcategories(total_cells)

In [12]:
len(total_cells)

30219

In [13]:
def reformatForName(cell_list):
    n_dict = {}
    intersects_g = 0
    nointersects_g = 0
    misfit_list = []
    for element in copy.deepcopy(cell_list):
        
        name = element.get('name')
        
        if name in n_dict:
            
            intersects = False
            for key in n_dict[name].keys():
                if not key in ['name', 'type', 'nameisid', 'nodelist', 'nameisunique']:
                    a = s2.CellUnion(map(lambda x: s2.CellId.from_token(x), element.get(key)))
                    b = s2.CellUnion(map(lambda x: s2.CellId.from_token(x), n_dict[name].get(key)))
                    if s2.CellUnion.get_intersection(a,b).cell_ids():
                        intersects = True
            if intersects:
                intersects_g += 1
                for key in n_dict[name].keys():
                    if not key in ['name', 'type', 'nameisid', 'nodelist', 'nameisunique']:
                        this_el = s2.CellUnion(map(lambda x: s2.CellId.from_token(x), element.get(key)))
                        dict_el = s2.CellUnion(map(lambda x: s2.CellId.from_token(x), n_dict[name].get(key)))
                        new_cell_list = list(map(lambda x: x.to_token(), get_union_preserve_custom(this_el, dict_el, key).cell_ids()))
                        n_dict[name][key] = new_cell_list
            else:
                nointersects_g += 1
#                 if len(misfit_list) < 30:
#                     print(element)
#                     print(n_dict[name])
                misfit_list.append(element)
        else:
            n_dict[name] = element

    print("intersects")
    print(intersects_g)
    print("no_intersects")
    print(nointersects_g)
    
    finallist = list(n_dict.values())
    for element in finallist:
        element['nameisunique'] = True
    for element in misfit_list:
        element['nameisunique'] = False
        n_dict[element['name']]['nameisunique'] = False
    return finallist + misfit_list

In [14]:
total_cells = reformatForName(total_cells)

intersects
855
no_intersects
1622


In [15]:
total_cells = reformatForName(total_cells)

intersects
0
no_intersects
1622


In [16]:
total_cells = reformatForName(total_cells)

intersects
0
no_intersects
1821


In [ ]:
total_cells = reformatForName(total_cells)

In [ ]:
total_cells = reformatForName(total_cells)

In [ ]:
len(total_cells)

In [16]:
with open('collegeonly_cells_v2.json', 'w') as write_file:
    json.dump(total_cells, write_file)

In [17]:
################

In [18]:
import json
import s2sphere as s2
import copy
import uuid

In [4]:
with open('new_total_cells_v7.json', 'r', encoding='utf8') as read_file:
    total_cells = json.load(read_file)

In [18]:
len(total_cells)

29364

In [19]:
def namecheck(cell_list, name):
    for cell in cell_list:
        if name in cell['name']:
            print(cell)

In [20]:
def namecheckallnames(cell_list, name):
    for cell in cell_list:
        for evalname in cell['allnames']:
            if name in evalname:
                print(cell)

In [21]:
def namecheckldict(ldict, name):
    for lindex, lkey in enumerate(ldict):
        cell = ldict[lkey]
        for evalname in cell['allnames']:
#             if lindex > 10:
#                 return
#             print(evalname)
            if name in evalname:
                print(cell)

In [22]:
def addIdIndex(cell_list):
    for index, element in enumerate(cell_list):
        element['addedindex'] = index
    return cell_list

In [23]:
total_cells = addIdIndex(total_cells)

In [24]:
namecheck(total_cells, 'University of California, Berkeley')

In [25]:
def get_union_preserve_custom(a,b,key):
    c = s2.CellUnion.get_union(a, b).cell_ids()
    running_union = s2.CellUnion()
    for cellId in copy.deepcopy(c):
        temp_union = s2.CellUnion([cellId])
        min_target = 10
        if '10' in key:
            min_target = 10
        elif '11' in key:
            min_target = 11
        elif '12' in key:
            min_target = 12
        elif '13' in key:
            min_target = 13
        elif '14' in key:
            min_target = 14
        elif '15' in key:
            min_target = 15
        elif '16' in key:
            min_target = 16
        if cellId.level() < min_target:
            temp_union = s2.CellUnion(temp_union.denormalize(min_target,1),raw=False)
        running_union = s2.CellUnion.get_union(temp_union, running_union,raw=False)
    return running_union

In [26]:
def reformatLocationDict(cell_list):
    l_dict = {}
    bad_count = 0
    for aIndex, element in enumerate(copy.deepcopy(cell_list)):
        if aIndex % 1000 == 0:
            print(aIndex)
            
        if 'allnames' not in element:
            element['allnames'] = ['%s' % element['name']]
                
        if 'alladdedindex' not in element:
            element['alladdedindex'] = [element['addedindex'] + 0]
            
        for key in ['name', 'type', 'nameisid', 'nodelist', 'cover10s', 'cover11s', 'cover12s', 'cover13s', 'cover14s', 'cover15s', 'cover10to16', 'cover10and11', 'cover11and12', 'cover12and13', 'cover13and14', 'cover14and15', 'cover15and16', 'nameisunique', 'addedindex']:
            element.pop(key, None)
                
        marklist = []
        
        #expansion not used atm proves intrusive
        #tomark16 = list(map(lambda x: x.to_token(), s2.CellUnion(map(lambda x: s2.CellId.from_token(x), element.get('cover16s')),raw=False).expand(16).cell_ids()))
        
        for mark_token in element.get('cover16s'):
            if mark_token in l_dict and mark_token not in marklist:
                marklist.append(mark_token)
                
        if marklist:
            bad_count += 1
            
            seen_elements = [element]
            
            #initialize with the current element so its usable in for loop
            this_el_16 = s2.CellUnion(map(lambda x: s2.CellId.from_token(x), element.get('cover16s')))
            
            for marked_tok in marklist:
                if l_dict[marked_tok] not in seen_elements:
                    seen_elements.append(l_dict[marked_tok])
                mark_el_16 = s2.CellUnion(map(lambda x: s2.CellId.from_token(x), l_dict[marked_tok].get('cover16s')))
                this_el_16 = get_union_preserve_custom(this_el_16, mark_el_16,'cover16s')
                
            consider_cell_list_16 = list(map(lambda x: x.to_token(), this_el_16.cell_ids()))
              
            
            control_cell_list_16 = None
            
            while control_cell_list_16 != consider_cell_list_16:
                control_cell_list_16 = copy.deepcopy(consider_cell_list_16)
                
                for token in consider_cell_list_16:
                    search_element = copy.deepcopy(l_dict.get(token))
                    if search_element and search_element not in seen_elements:
                        seen_elements.append(search_element)
                        search_el_16 = s2.CellUnion(map(lambda x: s2.CellId.from_token(x), search_element.get('cover16s')))
                        this_el_16 = get_union_preserve_custom(this_el_16, search_el_16,'cover16s')
                        
                consider_cell_list_16 = list(map(lambda x: x.to_token(), this_el_16.cell_ids()))
                        
            all_name_set = set()
            for name_el in seen_elements:
                for curr_name in name_el['allnames']:
                    all_name_set.add(curr_name)
            
            all_id_set = set()
            for id_el in seen_elements:
                for curr_id in id_el['alladdedindex']:
                    all_id_set.add(curr_id)
                
            new_using_element = {}
            new_using_element['cover16s'] = copy.deepcopy(consider_cell_list_16)
            new_using_element['allnames'] = copy.deepcopy(list(all_name_set))
            new_using_element['alladdedindex'] = copy.deepcopy(list(all_id_set))
            
            for token in new_using_element['cover16s']:
                l_dict[token] = new_using_element
        else:   
            for token in element.get('cover16s'):
                l_dict[token] = copy.deepcopy(element)
                
                

    print("badcount")
    print(bad_count)
    return l_dict

In [27]:
new_total_cells = reformatLocationDict(total_cells)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
badcount
4476


In [28]:
namecheckldict(new_total_cells, 'University of California, Berkeley')

In [29]:
len(new_total_cells)

184622

In [30]:
def reformatLocationDictByAddedIndex(cell_location_dict):
    indicesseen = set()
    reformatlist = []
    for tindex, token in enumerate(cell_location_dict):
        if(tindex) % 1000 == 0:
            print(tindex)
        element = cell_location_dict[token]
        if not set(element['alladdedindex']).issubset(indicesseen):
            reformatlist.append(element)
            indicesseen.update(set(element['alladdedindex']))    
    return reformatlist

In [31]:
total_cells = reformatLocationDictByAddedIndex(new_total_cells)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [32]:
with open('collegeonly_cells_v3.json', 'w', encoding='utf8') as write_file:
    json.dump(total_cells, write_file)

In [33]:
for element in new_total_cells1:
    count = 0
    if len(element['alladdedindex']) > 1:
        count += 1
print(count)

0


In [32]:
len(new_total_cells1)

448637

In [1]:
#################

In [1]:
import json
import s2sphere as s2
import copy
import uuid


In [2]:
total_cells[0]

NameError: name 'total_cells' is not defined

In [3]:
with open('collegeonly_cells_v3.json', 'r', encoding='utf8') as read_file:
    total_cells = json.load(read_file)

In [4]:
def get_union_preserve_custom(a,b,key):
    c = s2.CellUnion.get_union(a, b).cell_ids()
    running_union = s2.CellUnion()
    for cellId in copy.deepcopy(c):
        temp_union = s2.CellUnion([cellId])
        min_target = 10
        if '10' in key:
            min_target = 10
        elif '11' in key:
            min_target = 11
        elif '12' in key:
            min_target = 12
        elif '13' in key:
            min_target = 13
        elif '14' in key:
            min_target = 14
        elif '15' in key:
            min_target = 15
        elif '16' in key:
            min_target = 16
        if cellId.level() < min_target:
            temp_union = s2.CellUnion(temp_union.denormalize(min_target,1),raw=False)
        running_union = s2.CellUnion.get_union(temp_union, running_union,raw=False)
    return running_union

In [5]:
len(total_cells)

24645

In [6]:
def reformatForNameNew(cell_list):
    n_dict = {}
    intersects_g = 0
    nointersects_g = 0
    misfit_list = []
    good_list = []
    for theindex, element in enumerate(cell_list):
        if theindex % 1000 == 0:
            print(theindex)
            print(intersects_g)
            print(nointersects_g)
            print()
        
        isinlist = [checkname in n_dict for checkname in element.get('allnames')]
        if any(isinlist):
            intersectsname = element.get('allnames')[isinlist.index(True)]
            
            intersects = False
            for i in range(10, 17):
                
                a = s2.CellUnion(map(lambda x: s2.CellId.from_token(x), element.get('cover16s')))
                b = s2.CellUnion(map(lambda x: s2.CellId.from_token(x), n_dict[intersectsname].get('cover16s')))

                a = get_union_preserve_custom(a,a,'cover' + str(i) + 's')
                b = get_union_preserve_custom(b,b,'cover' + str(i) + 's')


                a.expand(i)
                b.expand(i)
                
                a.normalize()

                b.normalize()


                if s2.CellUnion.get_intersection(a,b).cell_ids():
                    intersects = True
            if intersects:
                
                intersects_g += 1
                this_el = s2.CellUnion(map(lambda x: s2.CellId.from_token(x), element.get('cover16s')))
                dict_el = s2.CellUnion(map(lambda x: s2.CellId.from_token(x), n_dict[intersectsname].get('cover16s')))
                new_cell_list = list(map(lambda x: x.to_token(), get_union_preserve_custom(this_el, dict_el, 'cover16s').cell_ids()))
                n_dict[intersectsname]['cover16s'] = new_cell_list
                
                
                nameset = set(element['allnames'])
                nameset.update(n_dict[intersectsname]['allnames'])
                
                n_dict[intersectsname]['allnames'] = list(nameset)
                
                indexset = set(element['alladdedindex'])
                indexset.update(n_dict[intersectsname]['alladdedindex'])
                
                n_dict[intersectsname]['alladdedindex'] = list(indexset)

            else:
                nointersects_g += 1
                misfit_list.append(element)
        else:
            good_list.append(element)
            for thisname in element.get('allnames'): 
                n_dict[thisname] = element

    print("intersects")
    print(intersects_g)
    print("no_intersects")
    print(nointersects_g)

    return good_list + misfit_list

In [7]:
total_cells1 = reformatForNameNew(total_cells)

0
0
0

1000
0
7

2000
0
14

3000
2
20

4000
4
32

5000
4
43

6000
6
55

7000
7
67

8000
8
75

9000
8
89

10000
9
96

11000
9
103

12000
11
114

13000
13
122

14000
14
132

15000
15
138

16000
15
163

17000
16
174

18000
16
185

19000
17
197

20000
17
203

21000
17
209

22000
17
215

23000
17
221

24000
99
789

intersects
180
no_intersects
1348


In [8]:
with open('collegeonly_v4.json', 'w', encoding='utf8') as write_file:
    json.dump(total_cells1, write_file)

In [ ]:
for element in total_cells1:
    if 'University of California, Berkeley' in element.get('allnames'):
        [print(el + ',') for el in element.get('cover16s')]
        print()

In [3]:
# [print(el + ',') for el in list(map(lambda x: x.to_token(), s2.CellUnion(g.get_edge_neighbors(), raw=False).cell_ids()))]

In [1]:
###################@@@@@

In [1]:
import json
import s2sphere as s2
import copy
import uuid
import math
from copy import deepcopy
import logging

In [2]:
lg = logging.getLogger()
lg.handlers = [] # This is the key thing for the question!

# Start defining and assigning your handlers here
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter("%(message)s")
handler.setFormatter(formatter)
lg.addHandler(handler)

In [3]:
with open('newusindex8.json', 'r', encoding='utf8') as read_file:
    total_cells = json.load(read_file)

In [4]:
len(total_cells)

6780

In [5]:
# def reformatToGeoJson(boundaries):
#     geojson = {
#       "type": "FeatureCollection"
#     }
    
#     geojson["features"] = []
    
#     for bindex,boundary in enumerate(boundaries):
        
#         newdict = {}
#         newdict["type"] = "Feature"
#         newdict["properties"] = {}
#         newdict["geometry"] = {}
#         newdict["geometry"]["type"] = "Polygon"
#         newdict["geometry"]["coordinates"] = [[]]
        
#         for theindex, thetuple in enumerate(boundary):
#             newdict["geometry"]["coordinates"][0].append(list(thetuple))
        
#         geojson["features"].append(newdict)
    

#     return geojson

In [6]:
# def find_all_boundary(parent_cell_union_list, level=16):
    
#     def find_islands(cell_union_list, level):
                
#         tight_union = s2.CellUnion(list(map(lambda x: s2.CellId.from_token(x), cell_union_list))).denormalize(level,1)

#         assert len(tight_union) > 0
    
#         all_islands = []
        
#         all_cells = set(map(lambda x: x.to_token(), tight_union[:]))
    
        
#         while all_cells:
#             all_cells_copy = all_cells.copy()
            
#             start = all_cells_copy.pop()
#             frontier = set([start])
#             all_nbrs = set([start])
            
#             while frontier:
#                 cell_id = frontier.pop()
#                 neighbors = list(map(lambda x: x.to_token(), s2.CellId.from_token(cell_id).get_all_neighbors(level)))
#                 for nbr in neighbors:
#                     if nbr in all_cells:
#                         if nbr in all_cells_copy and nbr not in frontier:
#                             frontier.add(nbr)
#                             all_cells_copy.remove(nbr)
#                             all_nbrs.add(nbr)

#             all_islands.append(list(all_nbrs))
            
#             for nbr in all_islands[-1]:
#                 all_cells.remove(nbr)
            
#         return all_islands
        
#     def findOuterBoundary(cell_union_island_list, level):
        
#         def algo(x):
#             return (math.atan2(x[0] - mlat, x[1] - mlng) + 2 * math.pi) % (2*math.pi)

#         def dist(x,y):
#             return math.hypot(y[0]-x[0],y[1]-x[1])

#         tight_union = s2.CellUnion(list(map(lambda x: s2.CellId.from_token(x), cell_union_island_list)))

#         tight_union_expanded = s2.CellUnion(list(map(lambda x: s2.CellId.from_token(x), cell_union_island_list)))
#         tight_union_expanded.expand(level)

#         tight_union_bound = tight_union.get_rect_bound()

#         topleft = s2.CellId.from_lat_lng(s2.LatLng.from_degrees(tight_union_bound.lat_hi().degrees, tight_union_bound.lng_lo().degrees)).parent(level)
#         topleftneighborbox = s2.CellUnion(list(topleft.get_all_neighbors(level)),raw=False).get_rect_bound()


#         topleft2 = s2.CellId.from_lat_lng(s2.LatLng.from_degrees(topleftneighborbox.lat_hi().degrees, topleftneighborbox.lng_lo().degrees)).parent(level)


#         bottomright = s2.CellId.from_lat_lng(s2.LatLng.from_degrees(tight_union_bound.lat_lo().degrees, tight_union_bound.lng_hi().degrees)).parent(level)
#         bottomrightneighborbox = s2.CellUnion(list(bottomright.get_all_neighbors(level)),raw=False).get_rect_bound()

#         bottomright2 = s2.CellId.from_lat_lng(s2.LatLng.from_degrees(bottomrightneighborbox.lat_lo().degrees, bottomrightneighborbox.lng_hi().degrees)).parent(level)



#         topleftpoint = s2.LatLng.from_degrees(s2.CellUnion([topleft2]).get_rect_bound().lat_hi().degrees, s2.CellUnion([topleft2]).get_rect_bound().lng_lo().degrees)
#         bottomrightpoint = s2.LatLng.from_degrees(s2.CellUnion([bottomright2]).get_rect_bound().lat_lo().degrees, s2.CellUnion([bottomright2]).get_rect_bound().lng_hi().degrees)


#         outerbound = s2.LatLngRect.from_point_pair(topleftpoint,bottomrightpoint)

#         rc = s2.RegionCoverer()

#         rc.max_level = level
#         rc.min_level = level
#         rc.level_mod = 1


#         outercovering = s2.CellUnion(list(rc.get_simple_reverse_covering(outerbound, tight_union, topleft2.to_point(), level)))

#         outercoveringexpanded = s2.CellUnion(list(rc.get_simple_reverse_covering(outerbound, tight_union, topleft2.to_point(), level)))

#         outercoveringexpanded.expand(level)


#         outercoveringexpanded.normalize()

#         outercovering.normalize()

#         tight_union_expanded.normalize()

#         tight_union.normalize()


#         finalintersectioninner = list(s2.CellUnion.get_intersection( outercoveringexpanded,tight_union).denormalize(level, 1))

#         finalintersectionouter = list(s2.CellUnion.get_intersection( outercovering,tight_union_expanded).denormalize(level, 1))


#         def all_possible_coordinates(forcoordinate_cellid_list):
#             thiscoordinateset = set()
            
# #             edges = set()
# #             diagonals = set()
            
#             for thiscellid in forcoordinate_cellid_list:
                
# #                 thiscelledges = set()
# #                 thiscelldiagonals = set()
                
#                 cellrep = s2.Cell(thiscellid)
                
#                 s2v0 = s2.LatLng.from_point(cellrep.get_vertex(0))
#                 s2v1 = s2.LatLng.from_point(cellrep.get_vertex(1))
#                 s2v2 = s2.LatLng.from_point(cellrep.get_vertex(2))
#                 s2v3 = s2.LatLng.from_point(cellrep.get_vertex(3))
                
#                 v0 = (s2v0.lat().degrees, s2v0.lng().degrees)
#                 v1 = (s2v1.lat().degrees, s2v1.lng().degrees)
#                 v2 = (s2v2.lat().degrees, s2v2.lng().degrees)
#                 v3 = (s2v3.lat().degrees, s2v3.lng().degrees)
                
#                 thiscoordinateset.update([v0,v1,v2,v3])
                
# #                 edges.update([(v0,v1), (v1,v0), (v0,v3), (v3,v0), (v1,v2), (v2,v1), (v2, v3), (v3, v2)])
                
# #                 diagonals.update([(v0,v2), (v2,v0), (v1,v3), (v3,v1)])
                
#             return thiscoordinateset
# #             return thiscoordinateset, edges, diagonals


        
#         outercoordinateset = all_possible_coordinates(finalintersectionouter)
        
#         innercoordinateset = all_possible_coordinates(finalintersectioninner)
# #         outercoordinateset, outeredges, outerdiagonals = all_possible_coordinates(finalintersectionouter)

# #         innercoordinateset, inneredges, innerdiagonals = all_possible_coordinates(finalintersectioninner)

#         finalcoordinatelist = list(innercoordinateset.intersection(outercoordinateset))

# #         combinededges = inneredges
# #         combineddiagonals = innerdiagonals
        
#         mlat = float(sum(x[0] for x in finalcoordinatelist)) / max(len(finalcoordinatelist), 1)
#         mlng = float(sum(x[1] for x in finalcoordinatelist)) / max(len(finalcoordinatelist), 1)
        
#         finalcoordinatelist.sort(key=algo)
        
    
# #         def undodiagonalmovements(combinededges, combineddiagonals, path):
            
# #             seencoords = set()
# #             frontier = [path[0]]
            
# # #             lg.warning(len(path))
            
# #             while frontier:
# #                 thepoint = frontier.pop()
# #                 indexofthepoint = path.index(thepoint)
# #                 if thepoint not in seencoords:
# #                     seencoords.add(thepoint)
# #                     reptuple = (thepoint, path[(indexofthepoint + 1) % len(path)])

# # #                     lg.warning(reptuple)

# #                     isdiagonal = False
# # #                     for diagonalset in combineddiagonals:
# #                     if reptuple in combineddiagonals:
# #                         isdiagonal = True
# # #                     lg.warning(isdiagonal)

# #                     if isdiagonal:
# #                         path[(indexofthepoint + 1) % len(path)], path[(indexofthepoint + 2) % len(path)] = path[(indexofthepoint + 2) % len(path)], path[(indexofthepoint + 1) % len(path)]
# #                     frontier.append(path[(indexofthepoint + 1) % len(path)])
# # #             lg.warning('\n')
# #             return path
            
        
        
#         originalpath = deepcopy(finalcoordinatelist)
        
# #         originalpath = undodiagonalmovements(combinededges, combineddiagonals, deepcopy(finalcoordinatelist))
        
        
#         if originalpath[0] != originalpath[-1]:
#             originalpath.append(originalpath[0])
        
#         originalcentroid = deepcopy((mlat, mlng))
        
#         for shiftindex in range(5):
                        
#             mlat = float(sum(x[0] for x in finalcoordinatelist)) / max(len(finalcoordinatelist), 1)
#             mlng = float(sum(x[1] for x in finalcoordinatelist)) / max(len(finalcoordinatelist), 1)
            
#             frontier = [finalcoordinatelist[shiftindex]]
#             newpath = [finalcoordinatelist[shiftindex]]
#             seencoords = set([finalcoordinatelist[shiftindex]])
#             while frontier:
#                 thepoint = frontier.pop()
#                 indexofthepoint = finalcoordinatelist.index(thepoint)

                
#                 horizon = 2

#                 horizonlist = []
#                 for horizonindex in range(1, horizon + 1):
#                     horizonpoint = finalcoordinatelist[(indexofthepoint + horizonindex) % len(finalcoordinatelist)]
#                     wrappedlastindex = ((indexofthepoint - 1) + len(finalcoordinatelist)) % (len(finalcoordinatelist))
                    
#                     if horizonpoint != finalcoordinatelist[indexofthepoint] or horizonpoint != finalcoordinatelist[wrappedlastindex]:
#                         horizonlist.append((dist(thepoint,horizonpoint), horizonpoint))

#                 thedist, closestpoint = min(horizonlist, key = lambda x: x[0])

#                 if closestpoint not in seencoords:
#                     frontier.append(closestpoint)
#                     newpath.append(closestpoint)
#                     seencoords.add(closestpoint)
                    
#             finalcoordinatelist = deepcopy(newpath)
            
#         flat = float(sum(x[0] for x in finalcoordinatelist)) / max(len(finalcoordinatelist), 1)
#         flng = float(sum(x[1] for x in finalcoordinatelist)) / max(len(finalcoordinatelist), 1)
        
#         finalcentroid = (flat, flng)
        
#         if not finalcoordinatelist[-1] == finalcoordinatelist[0]:
#             finalcoordinatelist.append(finalcoordinatelist[0])

#         return finalintersectioninner, finalintersectionouter, originalpath, originalcentroid, finalcoordinatelist, finalcentroid
#         #return finalcoordinatelist + [finalcoordinatelist[0]]

#     islandreturnlist = []
    
#     def reformatToSingleBoundaryOSM(boundary):
#         thestr = ''
#         thestr += '(('
#         for thetuple in boundary:
#             thestr += str(thetuple[1]) + ' ' + str(thetuple[0]) + ','
#         thestr = thestr[:-1]
#         thestr += ')),'
#         thestr = thestr[:-1]
#         return "MULTIPOLYGON(" + thestr + ")"
    
#     for island in find_islands(parent_cell_union_list, level):
#         cellboundaryinner, cellboundaryouter, originalpath, originalcentroid, finalpath, finalcentroid = findOuterBoundary(island, level)
        
#         osmooriginalpath = reformatToSingleBoundaryOSM(originalpath)
#         osmfinalpath = reformatToSingleBoundaryOSM(finalpath)
        
#         newdict = {}
        
#         newdict["island"] = island
#         newdict["cellboundaryinner"] = list(map(lambda x: x.to_token(), cellboundaryinner))
#         newdict["cellboundaryouter"] = list(map(lambda x: x.to_token(), cellboundaryouter))
#         newdict["originalpath"] = originalpath
#         newdict["originalcentroid"] = originalcentroid
#         newdict["osmooriginalpath"] = osmooriginalpath
#         newdict["finalpath"] = finalpath
#         newdict["finalcentroid"] = finalcentroid
#         newdict["osmfinalpath"] = osmfinalpath
        
#         islandreturnlist.append(newdict)
        
#     return islandreturnlist


In [2]:
import s2sphere as s2

In [3]:
s2.__path__

['/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/s2sphere']

In [7]:
def find_all_boundary(parent_cell_union_list, level=16):

    def find_islands(cell_union_list, level):

        tight_union = s2.CellUnion(list(map(lambda x: s2.CellId.from_token(x), cell_union_list))).denormalize(level,1)

        assert len(tight_union) > 0

        all_islands = []

        all_cells = set(map(lambda x: x.to_token(), tight_union[:]))


        while all_cells:
            all_cells_copy = all_cells.copy()

            start = all_cells_copy.pop()
            frontier = set([start])
            all_nbrs = set([start])

            while frontier:
                cell_id = frontier.pop()
                neighbors = list(map(lambda x: x.to_token(), s2.CellId.from_token(cell_id).get_edge_neighbors()))
                for nbr in neighbors:
                    if nbr in all_cells:
                        if nbr in all_cells_copy and nbr not in frontier:
                            frontier.add(nbr)
                            all_cells_copy.remove(nbr)
                            all_nbrs.add(nbr)

            all_islands.append(list(all_nbrs))

            for nbr in all_islands[-1]:
                all_cells.remove(nbr)

        return all_islands

    def findOuterBoundary(cell_union_island_list, level):

        def dist(x,y):
            return math.hypot(y[0]-x[0],y[1]-x[1])

        tight_union = s2.CellUnion(list(map(lambda x: s2.CellId.from_token(x), cell_union_island_list)))

        tight_union_expanded = s2.CellUnion(list(map(lambda x: s2.CellId.from_token(x), cell_union_island_list)))
        tight_union_expanded.expand(level)

        tight_union_bound = tight_union.get_rect_bound()

        topleft = s2.CellId.from_lat_lng(s2.LatLng.from_degrees(tight_union_bound.lat_hi().degrees, tight_union_bound.lng_lo().degrees)).parent(level)
        topleftneighborbox = s2.CellUnion(list(topleft.get_all_neighbors(level)),raw=False).get_rect_bound()


        topleft2 = s2.CellId.from_lat_lng(s2.LatLng.from_degrees(topleftneighborbox.lat_hi().degrees, topleftneighborbox.lng_lo().degrees)).parent(level)


        bottomright = s2.CellId.from_lat_lng(s2.LatLng.from_degrees(tight_union_bound.lat_lo().degrees, tight_union_bound.lng_hi().degrees)).parent(level)
        bottomrightneighborbox = s2.CellUnion(list(bottomright.get_all_neighbors(level)),raw=False).get_rect_bound()

        bottomright2 = s2.CellId.from_lat_lng(s2.LatLng.from_degrees(bottomrightneighborbox.lat_lo().degrees, bottomrightneighborbox.lng_hi().degrees)).parent(level)



        topleftpoint = s2.LatLng.from_degrees(s2.CellUnion([topleft2]).get_rect_bound().lat_hi().degrees, s2.CellUnion([topleft2]).get_rect_bound().lng_lo().degrees)
        bottomrightpoint = s2.LatLng.from_degrees(s2.CellUnion([bottomright2]).get_rect_bound().lat_lo().degrees, s2.CellUnion([bottomright2]).get_rect_bound().lng_hi().degrees)


        outerbound = s2.LatLngRect.from_point_pair(topleftpoint,bottomrightpoint)

        rc = s2.RegionCoverer()

        rc.max_level = level
        rc.min_level = level
        rc.level_mod = 1


        outercovering = s2.CellUnion(list(rc.get_simple_reverse_covering(outerbound, tight_union, topleft2.to_point(), level)))

        outercoveringexpanded = s2.CellUnion(list(rc.get_simple_reverse_covering(outerbound, tight_union, topleft2.to_point(), level)))

        outercoveringexpanded.expand(level)


        outercoveringexpanded.normalize()

        outercovering.normalize()

        tight_union_expanded.normalize()

        tight_union.normalize()


        finalintersectioninner = list(s2.CellUnion.get_intersection( outercoveringexpanded,tight_union).denormalize(level, 1))

        finalintersectionouter = list(s2.CellUnion.get_intersection( outercovering,tight_union_expanded).denormalize(level, 1))

        innertokens = set(list(map(lambda x: x.to_token(), finalintersectioninner)))
        outertokens = set(list(map(lambda x: x.to_token(), finalintersectionouter)))


        def checkccw(points):
            doubleArea = 0
            for indexp, point in enumerate(points):
                x1 = point[0]
                y1 = point[1]
                x2 = points[(indexp + 1) % len(points)][0]
                y2 = points[(indexp + 1) % len(points)][1]
                doubleArea += (x2 - x1) * (y2 + y1)
            return doubleArea

        def edgeconnections(forcoordinate_cellid_list, ccw=True, double=False):

            def algo(x):
                return (math.atan2(x[0] - mlat, x[1] - mlng) + 2 * math.pi) % (2*math.pi)

            edgeset = set()

            for thiscellid in forcoordinate_cellid_list:

                cellrep = s2.Cell(thiscellid)

                s2v0 = s2.LatLng.from_point(cellrep.get_vertex(0))
                s2v1 = s2.LatLng.from_point(cellrep.get_vertex(1))
                s2v2 = s2.LatLng.from_point(cellrep.get_vertex(2))
                s2v3 = s2.LatLng.from_point(cellrep.get_vertex(3))

                v0 = (s2v0.lat().degrees, s2v0.lng().degrees)
                v1 = (s2v1.lat().degrees, s2v1.lng().degrees)
                v2 = (s2v2.lat().degrees, s2v2.lng().degrees)
                v3 = (s2v3.lat().degrees, s2v3.lng().degrees)

                s2center = s2.LatLng.from_point(cellrep.get_center())

                mlat = s2center.lat().degrees
                mlng = s2center.lng().degrees

                edgeset.add((v0, v1))
                edgeset.add((v1, v0))

                edgeset.add((v0, v3))
                edgeset.add((v3, v0))

                edgeset.add((v1, v2))
                edgeset.add((v2, v1))

                edgeset.add((v3, v2))
                edgeset.add((v2, v3))

            return edgeset


        a = edgeconnections(finalintersectioninner,double=True,ccw=False)
        b = edgeconnections(finalintersectionouter,double=True,ccw=True)


        boundingedgeset = a.intersection(b)




        specificedge = boundingedgeset.pop()
        finalpath = [specificedge[0]]
        seencoords = set([specificedge[0]])
        boundingedgeset.add(specificedge)
        fresh = True

        edgecoorddict = {}
        for ttedge in boundingedgeset:
            if (ttedge[0][0], ttedge[0][1]) in edgecoorddict:
                edgecoorddict[(ttedge[0][0], ttedge[0][1])].add(ttedge)
            else:
                edgecoorddict[(ttedge[0][0], ttedge[0][1])] = set([ttedge])
            if (ttedge[1][0], ttedge[1][1]) in edgecoorddict:
                edgecoorddict[(ttedge[1][0], ttedge[1][1])].add(ttedge)
            else:
                edgecoorddict[(ttedge[1][0], ttedge[1][1])] = set([ttedge])


        while finalpath[0] != specificedge[0] or fresh:
            fresh = False

            boundingedgeset.remove(specificedge)

            #start of edge
            if specificedge[0] not in seencoords:
                finalpath.append(specificedge[0])
                seencoords.add(specificedge[0])

            coorspcoordset = edgecoorddict[(specificedge[1][0],specificedge[1][1])]

            copyset = coorspcoordset.copy()

            copyset.discard(specificedge)

            copyset.discard((specificedge[1], specificedge[0]))

            if len(copyset) == 1:
                specificedge = copyset.pop()
            elif len(copyset) == 2:
                cand1, cand2 = copyset.pop(), copyset.pop()
                if not cand1[1] == specificedge[1]:
                    specificedge = cand1
                elif not cand2[1] == specificedge[1]:
                    specificedge = cand2


        actualfinalpath = finalpath + [finalpath[0]]
        ccwarea = checkccw(actualfinalpath)


        if ccwarea <= 0:
            return finalintersectioninner, finalintersectionouter, actualfinalpath
        else:
            return finalintersectioninner, finalintersectionouter, actualfinalpath[::-1]

    def reformatToSingleBoundaryOSM(boundary):
        thestr = ''
        thestr += '(('
        for thetuple in boundary:
            thestr += str(thetuple[1]) + ' ' + str(thetuple[0]) + ','
        thestr = thestr[:-1]
        thestr += ')),'
        thestr = thestr[:-1]
        return "MULTIPOLYGON(" + thestr + ")"

    islandreturnlist = []

    for island in find_islands(parent_cell_union_list, level):
        cellboundaryinner, cellboundaryouter, ccwpath = findOuterBoundary(island, level)

        osmccwpath = reformatToSingleBoundaryOSM(ccwpath)


        newdict = {}

        newdict["island"] = island
        newdict["cellboundaryinner"] = list(map(lambda x: x.to_token(), cellboundaryinner))
        newdict["cellboundaryouter"] = list(map(lambda x: x.to_token(), cellboundaryouter))
        newdict["ccwpath"] = ccwpath
        newdict["osmccwpath"] = osmccwpath

        islandreturnlist.append(newdict)

    return islandreturnlist

In [8]:
def addBoundariesToList(dataset_list):
    copy_dataset_list = deepcopy(dataset_list)

    def reformatToOSM(boundaries):
        thestr = ''

        for boundary in boundaries:
            thestr += '(('
            for thetuple in boundary:
                thestr += str(thetuple[1]) + ' ' + str(thetuple[0]) + ','
            thestr = thestr[:-1]
            thestr += ')),'
        thestr = thestr[:-1]
        return "MULTIPOLYGON(" + thestr + ")"
    
    
    for itemindex, item in enumerate(copy_dataset_list):
        if itemindex % 1000 == 0:
            print(itemindex)
        if item.get('cover16s'):
            item["islands"] = find_all_boundary(item.get('cover16s'))
            completeoutercellboundary = set()
            completeinnercellboundary = set()
            completeccwpolygon = []     

            for island in item["islands"]: 

                completeoutercellboundary.update(island["cellboundaryouter"])
                completeinnercellboundary.update(island["cellboundaryinner"])

                completeccwpolygon.append(island["ccwpath"])

            newdict = {}

            osmcompleteccwpath = reformatToOSM(completeccwpolygon)

            newdict["ccwpath"] = completeccwpolygon

            newdict["cellboundaryinner"] = list(completeinnercellboundary)
            newdict["cellboundaryouter"] = list(completeoutercellboundary)

            newdict["osmccwpath"] = osmcompleteccwpath

            item["combinedislands"] = newdict
    return copy_dataset_list    

In [9]:
print(len(total_cells))

6780


In [10]:
splicelist = addBoundariesToList(total_cells)

0
1000
2000
3000
4000
5000
6000


In [ ]:
print(splicelist[0]["combinedislands"]['osmccwpath'])

In [12]:
with open('newusindex9pretty.json', 'w', encoding='utf8') as write_file:
    json.dump(splicelist, write_file, indent=4)

In [602]:
# splicelist[0]["cover16s"]

In [603]:
# _= [print(el + ',') for el in splicelist[0]["islands"][1]['cellboundaryouter']]

In [604]:
# splicelist[0].get('cover16s')

In [606]:
print(splicelist[0]["islands"][1]['originalpath'])

[(37.87447067704782, -122.22142497201747), (37.87566184724264, -122.22142497201747), (37.876419734256686, -122.22284344012643), (37.87685299766635, -122.22142497201747), (37.87717757598074, -122.22426188942785), (37.87761086033473, -122.22284344012643), (37.877935372416076, -122.22568031991976), (37.878368677713325, -122.22426188942785), (37.879126449803444, -122.22568031991976), (37.87988417660635, -122.22709873160005), (37.88031750741823, -122.22568031991976), (37.881075209875995, -122.22709873160005), (37.88064185812336, -122.22851712446668), (37.88183286704808, -122.22851712446668), (37.881399494355755, -122.2299354985176), (37.882590478935775, -122.2299354985176), (37.882157085304826, -122.23135385375079), (37.881723675307256, -122.23277219016417), (37.88291463097187, -122.23277219016417), (37.88248120003743, -122.23419050775568), (37.882047752738174, -122.23560880652335), (37.88323867948791, -122.23560880652335), (37.884429586465, -122.23560880652335), (37.88399611366001, -122.23

In [597]:
# yo = find_all_boundary(a)

In [213]:
# print(reformatToOSM(yo))

In [58]:
# print(json.dumps(reformatToGeoJson(yo)))

In [51]:
aa = s2.CellId.from_token('3b00c5875')

In [52]:
pp = s2.LatLng.from_point(s2.Cell(aa).get_vertex(0))

In [54]:
ss = pp.to_point()

In [ ]:
s2.CellId.from_lat_lng()

In [88]:
print(str(pp.lat().degrees) + ',' + str(pp.lng().degrees))

9.918144396037375,78.1108279156293


In [18]:
def alledgesanddiagonals(cell_list):
    
    def toUsableLatLng(s2p):
        latlngp = s2.LatLng.from_point(s2p)
        p = (latlngp.lat().degrees, latlngp.lng().degrees)
        return p

    def toS2pt(atuple):
        return s2.LatLng.from_degrees(atuple[0], atuple[1]).to_point()
    
    def midpointSet(p1, p2):  
        marginalerror = 0.00000001
        themidtuple = ((p1[0]+p2[0])/2, (p1[1]+p2[1])/2)
        v1 = (themidtuple[0] + marginalerror, themidtuple[1])
        v2 = (themidtuple[0] - marginalerror, themidtuple[1])
        v3 = (themidtuple[0], themidtuple[1] - marginalerror)
        v4 = (themidtuple[0], themidtuple[1] + marginalerror)
        return [v1, v2, v3, v4]

    edges = set()
    diagonals = set()
    
    for cell_id in cell_list:
        edge_union = s2.CellUnion(cell_id.get_edge_neighbors())
        for_vertex = s2.Cell(cell_id)
        s2connections = [(for_vertex.get_vertex(0), for_vertex.get_vertex(1)), (for_vertex.get_vertex(0), for_vertex.get_vertex(2)), (for_vertex.get_vertex(0), for_vertex.get_vertex(3)), (for_vertex.get_vertex(1), for_vertex.get_vertex(2)), (for_vertex.get_vertex(1), for_vertex.get_vertex(3)), (for_vertex.get_vertex(2), for_vertex.get_vertex(3))]
        latlngconnections = [(toUsableLatLng(connection[0]), toUsableLatLng(connection[1])) for connection in s2connections]
        midpointsets = [midpointSet(connection[0], connection[1]) for connection in latlngconnections]
        checking = [any([edge_union.contains(toS2pt(mdpt)) for mdpt in theset]) for theset in midpointsets]
        
        print(checking)
        
        for i, t_or_f in enumerate(checking):
            if t_or_f:
                edges.add((latlngconnections[i][0], latlngconnections[i][1]))
                edges.add((latlngconnections[i][1], latlngconnections[i][0]))
            else:
                diagonals.add((latlngconnections[i][0], latlngconnections[i][1]))
                diagonals.add((latlngconnections[i][1], latlngconnections[i][0]))
    return edges, diagonals
        

In [93]:
sss =s2.Cell(s2.CellId.from_token('47097baa5'))

In [94]:
sss.get_center()

Point: (0.6152166683865878, 0.15686615758776876, 0.772593981042408)

In [ ]:
alledgesanddiagonals([s2.CellId.from_token('a00000007'), s2.CellId.from_token('47097baa5')])

In [405]:
edgenbrscorners.difference(coordcorners)

{Point: (0.9999999989651402, 2.034536250313694e-05, 4.069134588798965e-05),
 Point: (0.9999999991721071, 0.0, 4.0691345896411415e-05),
 Point: (0.9999999995860662, 2.0345362515769904e-05, 2.0345362515769904e-05),
 Point: (0.9999999997930331, 0.0, 2.034536251998072e-05)}

In [388]:
coordcorners

{Point: (0.9999999989651402, 2.034536250313694e-05, 4.069134588798965e-05),
 Point: (0.9999999991721071, 0.0, 4.0691345896411415e-05),
 Point: (0.9999999995860662, 2.0345362515769904e-05, 2.0345362515769904e-05),
 Point: (0.9999999997930331, 0.0, 2.034536251998072e-05)}

In [357]:
f0.get_vertex(1)

Point: (0.5773659305790212, -0.577318945136258, -0.5773659305790212)

In [358]:
gg = f0.

In [351]:
for theface in [f0,f1,f2,f3,f4,f5,f6,f7]:
    print([dist(theface.get_vertex(0), theface.get_vertex(1)), dist(theface.get_vertex(0), theface.get_vertex(2)), dist(theface.get_vertex(0), theface.get_vertex(3)), dist(theface.get_vertex(1), theface.get_vertex(2)), dist(theface.get_vertex(1), theface.get_vertex(3)), dist(theface.get_vertex(2), theface.get_vertex(3))])
    print(theface.orientation())
    print()
    

[1.7510687064086356e-05, 1.7510883016982806e-05, 1.1074584459938347e-05, 1.1074809766996796e-05, 2.3492980276351833e-05, 1.75108295808397e-05]
1

[1.7510526772395676e-05, 1.7510758328043284e-05, 1.1074190140779485e-05, 1.1074415461623439e-05, 2.3492462494978592e-05, 1.75106692751899e-05]
0

[1.7510687064086356e-05, 1.107517592634038e-05, 1.751038427011923e-05, 1.7510384256564735e-05, 3.322372519689939e-05, 1.751068708538664e-05]
1

[1.1074584459938347e-05, 1.7510883016982806e-05, 1.7510687064086356e-05, 1.75108295808397e-05, 2.3492980276351833e-05, 1.1074809766996796e-05]
0

[1.1074190140779485e-05, 1.7510758328043284e-05, 1.7510526772395676e-05, 1.75106692751899e-05, 2.3492462494978592e-05, 1.1074415461623439e-05]
1

[1.751038427011923e-05, 1.107517592634038e-05, 1.7510687064086356e-05, 1.751068708538664e-05, 3.322372519689939e-05, 1.7510384256564735e-05]
0

[1.7510687064086356e-05, 1.107517592634038e-05, 1.751038427011923e-05, 1.7510384256564735e-05, 3.322372519689939e-05, 1.75106870

In [227]:
def dist(x,y):
    return math.hypot(y[0]-x[0],y[1]-x[1])

In [321]:
f7.face()

7

In [203]:
bbb.face()

2

In [204]:
[dist(bba.get_vertex(0), bba.get_vertex(1)), dist(bba.get_vertex(0), bba.get_vertex(2)), dist(bba.get_vertex(0), bba.get_vertex(3)), dist(bba.get_vertex(1), bba.get_vertex(2)), dist(bba.get_vertex(1), bba.get_vertex(3)), dist(bba.get_vertex(2), bba.get_vertex(3))]


[1.732533929229046e-05,
 2.0964529538994757e-05,
 1.9872674188048066e-05,
 1.9872995603198868e-05,
 3.08330805243693e-05,
 1.7325237034315364e-05]

In [205]:
[dist(bbb.get_vertex(0), bbb.get_vertex(1)), dist(bbb.get_vertex(0), bbb.get_vertex(2)), dist(bbb.get_vertex(0), bbb.get_vertex(3)), dist(bbb.get_vertex(1), bbb.get_vertex(2)), dist(bbb.get_vertex(1), bbb.get_vertex(3)), dist(bbb.get_vertex(2), bbb.get_vertex(3))]


[1.7325441555519647e-05,
 2.0964236144504224e-05,
 1.9872882739906703e-05,
 1.987320414803462e-05,
 3.0833663761610875e-05,
 1.732533929229046e-05]

In [206]:
[dist(aaa.get_vertex(0), aaa.get_vertex(1)), dist(aaa.get_vertex(0), aaa.get_vertex(2)), dist(aaa.get_vertex(0), aaa.get_vertex(3)), dist(aaa.get_vertex(1), aaa.get_vertex(2)), dist(aaa.get_vertex(1), aaa.get_vertex(3)), dist(aaa.get_vertex(2), aaa.get_vertex(3))]


[1.751038427011923e-05,
 1.107517592634038e-05,
 1.7510687064086356e-05,
 1.751068708538664e-05,
 3.322372519689939e-05,
 1.7510384256564735e-05]

In [207]:
[dist(aab.get_vertex(0), aab.get_vertex(1)), dist(aab.get_vertex(0), aab.get_vertex(2)), dist(aab.get_vertex(0), aab.get_vertex(3)), dist(aab.get_vertex(1), aab.get_vertex(2)), dist(aab.get_vertex(1), aab.get_vertex(3)), dist(aab.get_vertex(2), aab.get_vertex(3))]


[1.7510384268232458e-05,
 1.1074809779652416e-05,
 1.7510384268232458e-05,
 1.751038427011923e-05,
 3.3223528069145794e-05,
 1.751038427011923e-05]